# 3. Text preprocessing

## Setup

In [1]:
%run __init__.py

In [2]:
import pandas as pd

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Common pipeline

## COVID-19
bla bla bla

### Loading the dataframe

In [ ]:
CORD_DATASET_DIR = os.path.join(DATA_DIR, 'cord19')
CORD19_FILE_PATH = os.path.join(CORD_DATASET_DIR, 'cord19_dataframe.pkl')

cord19_df = pd.read_pickle(CORD19_FILE_PATH)

## Agriculture

### Loading the dataframe

In [3]:
AGRICULTURE_DATASET_DIR = os.path.join(DATA_DIR, 'agriculture')
PMC_FILE_PATH = os.path.join(AGRICULTURE_DATASET_DIR, 'pmc_dataframe.pkl')

pmc_df = pd.read_pickle(PMC_FILE_PATH)

### Preprocessing text

In [4]:
publications = pmc_df['text_cleaned'].values

In [5]:
import numpy as np
import multiprocessing as mp

import string
import spacy 
import en_core_web_sm
from sklearn.base import TransformerMixin, BaseEstimator

nlp = en_core_web_sm.load()
nlp.Defaults.stop_words |= {"et","al", "introduction", "Fig", "fig", "figure"}

class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, n_jobs=1):
        self.n_jobs = n_jobs

    def fit(self, X, y=None):
        return self

    def transform(self, X, *args, **kwargs):
        return [self._preprocess_text(text) for text in X]

    def _preprocess_part(self, part):
        return part.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        doc = nlp(text)
        return [t.lemma_ for t in doc if len(t.text) > 2 and
                not t.is_stop and t.text not in string.punctuation
                and t.is_alpha and not t.is_digit]

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if not t.is_stop]
    
    def _remove_digits(self, doc):
        return [t for t in doc if t.is_alpha and not t.is_digit]

    def _lemmatize(self, doc):
        return [t.lemma_ for t in doc if len(t.text) > 2]
    

### LDA

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

def dummy(doc):
    return doc

lda_pipeline = Pipeline([('preprocessing', TextPreprocessor()),
                     ('vectorizer', CountVectorizer(preprocessor=dummy, tokenizer=dummy)),
                     ('model', LatentDirichletAllocation())])


In [33]:
from sklearn.model_selection import RandomizedSearchCV

search_params = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'model__n_components': [10, 15, 20, 25, 30],
    'model__learning_decay': [.5, .7, .9]
}

search = RandomizedSearchCV(lda_pipeline, param_distributions=search_params, n_iter=15)
search.fit(publications)

KeyboardInterrupt: 

In [ ]:
best_lda_model = search.best_estimator_

print(f"Best pipeline parameters: {search.best_params_}")
print(f"Best Log Likelihood Score: {search.best_score_}")

In [37]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()



Visualization. Do this with the best model obtained before.

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

In [53]:
# {'vectorizer__ngram_range': (1, 1), 'model__n_components': 10, 'model__learning_decay': 0.7}

preprocessed_text = TextPreprocessor().fit_transform(publications)
tf_vectorizer = CountVectorizer(preprocessor=dummy, tokenizer=dummy, ngram_range=(1, 1))
dtm_tf = tf_vectorizer.fit_transform(preprocessed_text)
lda_tf = LatentDirichletAllocation(n_components=10, learning_decay=0.7, random_state=42).fit(dtm_tf)

In [54]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda_tf, tf_feature_names, 10)

Topic #0: plant gene stress expression transgenic protein control line show level
Topic #1: plant soil rice endophyte treatment root sample study increase high
Topic #2: plant gene expression level show protein leave mutant acid type
Topic #3: plant heat metabolite stress ABA light high group temperature yield
Topic #4: plant stress increase soil water rice root high level activity
Topic #5: crop system soil yield base practice high food rice study
Topic #6: genotype gliadin high group LABE show type protein osteoclast modern
Topic #7: plant medicinal take informant decoction Herb orally MED Wild report
Topic #8: plant system food gene sequence fungus fagaceous fungal specie sample
Topic #9: plant growth root cell phytolith sensor node different acid Arabidopsis



In [55]:
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim

metric_coherence_gensim(measure='c_v', 
                        top_n=20, 
                        topic_word_distrib=lda_tf.components_, 
                        dtm=dtm_tf, 
                        vocab=np.array([x for x in tf_vectorizer.vocabulary_.keys()]), 
                        texts=preprocessed_text,
                        return_mean=True)

0.7455770267958253

In [ ]:
def coherence_score(estimator, X, y):
    metric_coherence_gensim(measure='c_v', 
                            top_n=15, 
                            topic_word_distrib=lda_tf.components_, 
                            dtm=dtm_tf, 
                            vocab=np.array([x for x in tf_vectorizer.vocabulary_.keys()]), 
                            texts=preprocessed_text,
                            return_mean=True)

In [57]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1  24.213775
5       0.648955  71.707100       2        1  17.927704
4      71.019684   9.971006       3        1  15.551041
1       3.785981  33.185135       4        1   9.410253
2     -42.666500  -9.222070       5        1   8.198189
7      -3.621689 -43.526516       6        1   6.926033
9      -3.632402  -4.171708       7        1   6.777989
8     -36.584400  35.804413       8        1   5.415483
3      44.391380  50.615383       9        1   4.002947
6      40.235809 -31.591665      10        1   1.576586, topic_info=                Term          Freq         Total Category  logprob  loglift
23617          plant  13810.000000  13810.000000  Default  30.0000  30.0000
26083           soil   2428.000000   2428.000000  Default  29.0000  29.0000
26512         stress   2738.000000   2738.000000  Default  28.0000  28.0000
25186           rice   1933.000000   1933.000000  Default  27.0000  27.0000
15958           crop   1807.000000   1807.000000  Default  26.0000  26.0000
...              ...           ...           ...      ...      ...      ...
15958           crop     33.055454   1807.590930  Topic10  -5.4771   0.1483
16677     difference     30.121462    995.805775  Topic10  -5.5701   0.6516
25898  significantly     28.980949    863.157255  Topic10  -5.6087   0.7560
20925            low     28.951303   1223.885161  Topic10  -5.6097   0.4057
25604         season     26.731390    527.208910  Topic10  -5.6895   1.1682

[750 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
11         1  0.118635    ABA
11         3  0.212105    ABA
11         5  0.104255    ABA
11         7  0.039545    ABA
11         9  0.521275    ABA
...      ...       ...    ...
28677      9  0.096597  yield
28677     10  0.016719  yield
28708      1  0.061999   zinc
28708      3  0.020666   zinc
28708      4  0.916205   zinc

[2141 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 5, 2, 3, 8, 10, 9, 4, 7])

### Latent Semantic Analysis (LSA)

In [16]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

lsa_pipeline = Pipeline([('preprocessing', TextPreprocessor()),
                 ('vectorizer', TfidfVectorizer(preprocessor=dummy, tokenizer=dummy)),
                 ('model', TruncatedSVD(n_components=10))])

In [21]:
tf_feature_names = lsa_pipeline['vectorizer'].get_feature_names()
print_top_words(lsa_pipeline['model'], tf_feature_names, 10)

Topic #0: plant stress gene rice soil expression transgenic root increase protein
Topic #1: rice soil crop season emission residue system farm farmer winter
Topic #2: food farmer farm system household farming livestock crop climate diversity
Topic #3: stress heat salt tolerance drought NaCl food ROS salinity ALDH
Topic #4: transgenic gene rice expression emission crop line season protein drought
Topic #5: medicinal salt heat yield plant stress informant NaCl hypertension season
Topic #6: transgenic salt rice sensor food NaCl iot emission node system
Topic #7: emission sensor rice node season iot temperature heat defense Las
Topic #8: medicinal gene mutant informant hypertension wild decoction hydrophilin protein RNAi
Topic #9: endophyte yield sensor node grain root iot bean soil irrigation



In [30]:
lsa_search_params = {
    'vectorizer__min_df': [1, 2, 3],
    'vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2), (1, 3), (2, 3)],
    'model__n_components': [5, 10, 15, 20, 25, 30],
    'model__algorithm': ["arpack", "randomized"]
}

lsa_search = RandomizedSearchCV(lsa_pipeline, param_distributions=lsa_search_params, n_iter=2)
lsa_search.fit(publications)

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator Pipeline(steps=[('preprocessing', TextPreprocessor()),
                ('vectorizer',
                 TfidfVectorizer(preprocessor=<function dummy at 0x7f01195fd7b8>,
                                 tokenizer=<function dummy at 0x7f01195fd7b8>)),
                ('model', TruncatedSVD(n_components=10))]) does not.

Visualization

In [ ]:
tf_feature_names = lsa_pipeline['vectorizer'].get_feature_names()
print_top_words(lsa_pipeline['model'], tf_feature_names, 10)

### NMF

In [58]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

nmf_pipeline = Pipeline([('preprocessing', TextPreprocessor()),
                     ('vectorizer', TfidfVectorizer(preprocessor=dummy, tokenizer=dummy)),
                     ('model', NMF(n_components=10))])

In [59]:
nmf_pipeline.fit_transform(publications)

array([[2.42916162e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.21219871e-01, 0.00000000e+00, 2.00045046e-03, ...,
        2.22969065e-04, 1.59647843e-01, 1.28742630e-01],
       [1.83559488e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [8.08683820e-03, 0.00000000e+00, 1.54996251e-02, ...,
        1.37068304e-02, 3.32296836e-02, 1.96712342e-03],
       [0.00000000e+00, 5.82704255e-03, 9.87629230e-04, ...,
        7.81970521e-03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.52979793e-02, 0.00000000e+00, ...,
        1.02506757e-02, 1.82583053e-02, 1.56927405e-01]])

In [23]:
tf_feature_names = nmf_pipeline['vectorizer'].get_feature_names()
print_top_words(nmf_pipeline['model'], tf_feature_names, 10)

Topic #0: plant endophyte defense herbivore metabolite volatile GSL insect touch expression
Topic #1: rice soil emission season crop residue paddy straw winter treatment
Topic #2: food farmer farm system household CSA climate farming crop livestock
Topic #3: stress plant salt heat tolerance NaCl drought gene expression treatment
Topic #4: transgenic plant gene expression line protein sequence transcript overexpressor RNAi
Topic #5: metal toxicity soil plant root concentration heavy acid uptake increase
Topic #6: sensor iot node IoT wireless network communication Smart power Campus
Topic #7: plant infect CMV Las virus infected infection NahG citri symptomatic
Topic #8: medicinal plant informant hypertension decoction Herb traditional specie orally ailment
Topic #9: crop yield soil legume grain bird bean habitat residue wheat



In [ ]:
nmf_pip